In [ ]:
# Libraries import

import os
import cv2
import numpy as np
import joblib  # For saving/loading the SVM model
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt


In [ ]:
# Constants
IMAGE_SIZE = (128, 128)  # Resize all images
N_COMPONENTS = 500       # PCA dimensionality
NU_VALUE = 0.02          # SVM sensitivity (lower = stricter detection)


In [ ]:
#Load and Preprocess a Single Image

def load_and_preprocess_image(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"❌ Warning: Unable to read {image_path}")
            return None
        img = cv2.resize(img, IMAGE_SIZE)
        img = img.astype(np.float32) / 255.0
        return img.flatten()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None



In [ ]:
#Load and Preprocess All Images in a Folder

def load_images_from_folder(folder):
    images, filenames = [], []
    
    if not os.path.exists(folder):
        print(f"❌ Error: Folder '{folder}' not found!")
        return np.array(images), filenames

    file_list = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith((".png", ".jpg"))]

    if not file_list:
        print(f"⚠️ Warning: No images found in '{folder}'")
        return np.array(images), filenames

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_and_preprocess_image, file_list))

    for i, result in enumerate(results):
        if result is not None:
            images.append(result)
            filenames.append(os.path.basename(file_list[i]))

    return np.array(images), filenames


In [ ]:
# Train One-Class SVM

def train_one_class_svm(normal_folder, model_path="one_class_svm_model.pkl"):
    print("\n📂 Loading normal images for training...")
    X_train, _ = load_images_from_folder(normal_folder)

    if X_train.shape[0] == 0:
        print("❌ Error: No training images found!")
        return

    print(f"✅ Loaded {X_train.shape[0]} images for training. Feature size: {X_train.shape[1]}")

    print("\n🧠 Applying PCA...")
    pca = PCA(n_components=min(N_COMPONENTS, X_train.shape[1]))
    X_train_pca = pca.fit_transform(X_train)

    print("\n🚀 Training One-Class SVM...")
    svm_model = OneClassSVM(kernel="rbf", gamma="auto", nu=NU_VALUE)
    svm_model.fit(X_train_pca)

    joblib.dump((svm_model, pca), model_path)
    print(f"✅ Model saved at {model_path}")


In [ ]:
#Test One-Class SVM and Save Results

def test_one_class_svm(test_folder, model_path="one_class_svm_model.pkl", output_csv="anomaly_results.csv"):
    if not os.path.exists(model_path):
        print("❌ Error: Model file not found!")
        return

    print("\n📂 Loading test images...")
    X_test, filenames = load_images_from_folder(test_folder)

    if X_test.shape[0] == 0:
        print("❌ No test images found!")
        return

    print(f"✅ Loaded {X_test.shape[0]} test images.")

    svm_model, pca = joblib.load(model_path)
    X_test_pca = pca.transform(X_test)

    print("\n🔍 Predicting anomalies...")
    predictions = svm_model.predict(X_test_pca)

    results = pd.DataFrame({"Filename": filenames, "Prediction": predictions})
    results["Prediction"] = results["Prediction"].map({1: "Normal ✅", -1: "Anomaly ❌"})
    results.to_csv(output_csv, index=False)

    print(f"\n📄 Results saved to {output_csv}")
    print("\n🔹 Anomaly Detection Summary:")
    print(results.value_counts("Prediction"))
    print("\n✅ Detection completed!")


In [ ]:
#Main Execution

# Paths
normal_data_folder = "One_Class_SVM/Normal"
test_data_folder = "One_Class_SVM/Attack"
svm_model_path = "One_Class_SVM/one_class_svm_model.pkl"
output_results_csv = "One_Class_SVM/anomaly_results.csv"

# Train and test
train_one_class_svm(normal_data_folder, svm_model_path)
test_one_class_svm(test_data_folder, svm_model_path, output_results_csv)


In [ ]:
# Load results
results = pd.read_csv(output_results_csv)
results.head()
